Global Pruning Method on Node Classification Task of Cora Dataset
----------------------------

## Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import os.path as osp
import torch
import torch.nn.functional as F
import torch.nn as nn
import time
import copy
import psutil
import itertools
import tracemalloc
import gc
from torch.optim import Adam
import statistics as stat

import sys
import argparse
import os
import os.path as osp
import numpy as np
import random



import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv
from torch_geometric.typing import WITH_TORCH_SPLINE_CONV

import torch.nn.utils.prune as prune
from torch.nn.utils.prune import global_unstructured, L1Unstructured

### Sparsity

- The sparsity is the parameter that is determines the rate of pruning across the layer. It is a value in range(0,0.1,1). This parameter is fixed for this notebook and change for remaining experiment. We determine it before training process. Here is all values of sparsities.

In [2]:
0.1, 0.2, 0.3, 0.4,  0.5, 0.6 , 0.7,  0.8, 0.9
;

''

### Pruning function and fuction for loading pruned model

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.prune import global_unstructured, L1Unstructured, remove

def global_pruning_torch(model, parameters_to_prune, sparsity, path):
    # Pruning the model layers globally using PyTorch library
    global_unstructured(parameters_to_prune, pruning_method=L1Unstructured, amount=sparsity)
    
    # Removing pruning reparameterization to save only the final weights
    for module, name in parameters_to_prune:
        remove(module, name)
    
    # Convert the pruned model to sparse model
    #pruned_model = SparseModel(model)
    state = state_sparse_model(model, eval_acc=None)
    torch.save(state, path)
    
    return model


    
def state_sparse_model(model, eval_acc):
    '''
    This funcrion Removes Zeroed Weights
    and saves non-zero weights as state dictionary
    '''
    state_dict = model.state_dict()
    non_zero_state = {
        k: v.to_sparse() if torch.count_nonzero(v) < v.numel() else v
        for k, v in state_dict.items()
    }
    non_zero_state_dict = {'net': non_zero_state, 'epoch': epoch, 'acc': eval_acc}
    return non_zero_state_dict
    

    
def load_sparse_model(state_path, original_model):
    """
    This function provides a practical approach to loading sparse models into dense environments,
    offering a good balance between memory efficiency and model functionality.
    """
    non_zero_state_dict = torch.load(state_path)
    sparse_model_state = original_model.state_dict()

    for k, v in non_zero_state_dict['net'].items():
        if isinstance(v, torch.Tensor) and v.is_sparse:
            sparse_model_state[k] = v.to_dense()
        else:
            sparse_model_state[k] = v

    original_model.load_state_dict(sparse_model_state, strict=False)
    

    return original_model


def load_and_evaluate_pruned_model(model_path):
    """
    This function loads the pruned model from disk and evaluates it.
    """
    # Instantiate the model
    model = Net()
    
    # Load the pruned model
    sparse_model = load_sparse_model(model_path, model)
    print("Pruned model loaded.")
    

    return sparse_model
    

In [3]:
## New version

import torch
import torch.nn as nn
from torch.nn.utils.prune import global_unstructured, L1Unstructured, remove

def global_pruning_torch(model, parameters_to_prune, sparsity, path):
    # Pruning the model layers globally using PyTorch library
    global_unstructured(parameters_to_prune, pruning_method=L1Unstructured, amount=sparsity)
    
    # Removing pruning reparameterization to save only the final weights
    for module, name in parameters_to_prune:
        remove(module, name)
        
    #pth_name = f"pure_pruned_model.pth"   
    #ckpt_pruned_path = os.path.join(ckpt_dir, pth_name)    
    #state=model.state_dict()
    #torch.save(state, ckpt_pruned_path)
    #pure_pruned_model_size = os.path.getsize(ckpt_pruned_path)
    #print(f"pure_pruned_model_size:{pure_pruned_model_size}") 
    
    
    state = state_sparse_model(model, eval_acc=None)
    torch.save(state, path)
    pruned_model_size = os.path.getsize(path)
    #print(f"pruned_model_size:{pruned_model_size}")

def state_sparse_model(model, eval_acc=None, epoch=None):
    state_dict = model.state_dict()
    compressed_state = {}
    
    for k, v in state_dict.items():
        if torch.is_tensor(v):
            mask = v != 0
            if mask.any():  # Only compress if there are non-zeros
                compressed_state[k] = {
                    'shape': v.shape,
                    'values': v[mask]  # Store only non-zero values
                }
            else:
                compressed_state[k] = v  # Keep original if all zeros
        else:
            compressed_state[k] = v
    
    return {'net': compressed_state, 'epoch': epoch, 'acc': eval_acc}

def load_sparse_model(state_path, original_model):
    """
    Loads a model saved in the custom compressed format (non-zero values only).
    Reconstructs dense tensors before loading into the model.
    """
    # Load the compressed state_dict
    compressed_state = torch.load(state_path)
    compressed_weights = compressed_state['net']
    
    # Initialize a new state_dict for the original model
    new_state_dict = original_model.state_dict()
    
    for k, v in compressed_weights.items():
        if isinstance(v, dict) and 'shape' in v and 'values' in v:
            # Reconstruct dense tensor from compressed format
            dense_tensor = torch.zeros(v['shape'], dtype=v['values'].dtype)
            mask = (dense_tensor != 0)  # All False initially
            # We need to know the positions of non-zero values (if available)
            # If indices were saved, use them; otherwise, assume sequential filling (simpler but may not match original positions)
            if 'indices' in v:
                # If you saved indices (advanced version)
                dense_tensor[v['indices']] = v['values']
            else:
                # If only values were saved (simpler version)
                # Flatten and fill non-zeros sequentially (may not match original positions)
                flat_tensor = dense_tensor.view(-1)
                flat_tensor[:len(v['values'])] = v['values']
                dense_tensor = flat_tensor.reshape(v['shape'])
            
            new_state_dict[k] = dense_tensor
        else:
            # If it's a normal tensor (e.g., biases, batch norm stats)
            new_state_dict[k] = v
    
    # Load the reconstructed state_dict
    original_model.load_state_dict(new_state_dict, strict=False)
    return original_model
    



def load_and_evaluate_pruned_model(model_path):
    """
    This function loads the pruned model from disk and evaluates it.
    """
    # Instantiate the model
    model = Net()
    
    # Load the pruned model
    sparse_model = load_sparse_model(model_path, model)
    print("Pruned model loaded.")
    

    return sparse_model


### Functions for Mmeasuring criterias

In [4]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)

# The model size based on the number of parameters
def get_model_size_param(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width


def get_model_sparsity(model: nn.Module) -> float:
    ''' 
    The input is layers of pruned model and the output is the sparsity after pruning.
    '''
    Sparsity=dict()
    global_zero=0
    global_nzero=0
    layyers=[]
    spars=[]
    for name, param in model.named_parameters(): 
        if 'weight' in name:
                    zero=float(torch.sum(param == 0))
                    nzero=float(param.nelement())
                    sparsity=  float(zero)/ float(nzero)
                    print( f'Sparsity in {name}: {sparsity:.3f}' )
                    layyers.append(name)
                    spars.append(sparsity)
                    global_zero +=zero
                    global_nzero +=nzero



    Sparsity={key: value for key, value in zip(layyers,spars)}
    global_sparsity= float(global_zero) /float(global_nzero)
    Sparsity.update({'Global sparsity':  global_sparsity})
    print("Global sparsity: {:.3f}".format(global_sparsity))
    return   Sparsity   

### Arguments

In [5]:


# Clearing the arguments
sys.argv = ['']
parser = argparse.ArgumentParser()
parser.add_argument('--model_name',type=str,default='Spline')
parser.add_argument('--dataset_name',type=str,default='Cora')
parser.add_argument('--num_deg',type=int,default=1000)
parser.add_argument('--num_layers', type=int, default=2)
parser.add_argument('--hidden_units',type=int,default=16)
parser.add_argument('--max_epoch',type=int,default=50)
parser.add_argument('--max_cycle',type=int,default=10)
parser.add_argument('--weight_decay',type=float,default=0)
parser.add_argument('--lr',type=float,default=0.001)
###############################################################


args = parser.parse_args()
print(args)

Namespace(model_name='Spline', dataset_name='Cora', num_deg=1000, num_layers=2, hidden_units=16, max_epoch=50, max_cycle=10, weight_decay=0, lr=0.001)


### save path for model

In [6]:

if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
if not os.path.isdir(os.path.join('checkpoint', f"Cora")):
    os.mkdir(os.path.join('checkpoint', f"Cora"))
ckpt_dir = f"./checkpoint/Cora/"


def save_best(ckpt_dir, epoch, state,  model_name, eval_acc, is_best, is_pruned):
    print('saving....')
    #model.to_device()
    state_save = {
        'net':state,
        'epoch':epoch,
        'acc': eval_acc 
        }
    best_pth_name = f'{args.model_name}_best.pth'
    fine_tuned_pth_name = f'{args.model_name}_fine_tuned_best.pth'
  
    if is_pruned & is_best:
        ckpt_path = os.path.join(ckpt_dir, fine_tuned_pth_name) 
        torch.save(state_save, ckpt_path)
    
     
    if is_pruned== False & is_best:
        ckpt_path = os.path.join(ckpt_dir, best_pth_name)  
        torch.save(state_save, ckpt_path)
           
        
    model.to(device)

## Loading Dataset

In [7]:

if not WITH_TORCH_SPLINE_CONV:
    quit("This example requires 'torch-spline-conv'")

dataset = args.dataset_name
transform = T.Compose([
    T.RandomNodeSplit(num_val=500, num_test=500),
    T.TargetIndegree(),
])
path =  'data'
dataset = Planetoid(path, dataset, transform=transform)
data = dataset[0]

### Model for training

In [8]:


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(dataset.num_features, 16, dim=1, kernel_size=2)
        self.conv2 = SplineConv(16, dataset.num_classes, dim=1, kernel_size=2)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

### Function for training

In [9]:


def train(callbacks = None):
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
    if callbacks is not None:
        for callback in callbacks:
                callback()


                


def evaluate_Spline(model, mask):
    model.eval()
    for _, mask in mask:
        log_probs, accs = model(data), []
        # calculate acc
        pred = log_probs[mask].max(1)[1]
        acc=pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        # Calculate loss
        loss=F.nll_loss(model(data)[mask], data.y[mask])
    
    eval_state = {'loss': loss / (mask.sum().item()), 'acc': acc}
    return eval_state



@torch.no_grad()
def test(model):
    model.eval()
    log_probs, accs = model(data), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

### Implementing Global Pruning the Model and Re-Evaluate the Accuracy.
 - First, we determine the candidate layers for pruning. 

In [10]:
for name, param in model.named_parameters():
  
        print(name)

conv1.weight
conv1.bias
conv1.lin.weight
conv2.weight
conv2.bias
conv2.lin.weight


### Criteria for measurements

In [11]:

parameters_to_prune = [
    (model.conv1, 'weight'),
     (model.conv1.lin, 'weight'),
    (model.conv2, 'weight'),
     (model.conv2.lin, 'weight'),
]

In [15]:
### Setting Sparsity
sparsity=0.1

# The number of epochs 
num_epochs=5
# The number of iterations
num_iterations=2

In [16]:

# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
Eva_final=dict()

# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.

#Base model
Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]
Base_Energy_Consumption=[]
Base_Cpu_Usage=[]
Base_Memory_Usage=[]

#Pruned model
Pruned_model_accuracy=[]
T_pruned_model=[]
Num_parm_pruned_model=[]
Pruned_model_size=[]
Pruned_Energy_Consumption=[]
Pruned_Cpu_Usage=[]
Pruned_Memory_Usage=[]

#Pruned and finetune model
Pruned_finetune_model_accuracy=[]
T_pruned_finetune_model=[]
Num_parm_pruned_finetune_model=[]
Pruned_finetune_model_size=[]
Pruned_finetune_Energy_Consumption=[]
Pruned_finetune_Cpu_Usage=[]
Pruned_finetune_Memory_Usage=[]

#recording sparsities of layers
Spar_conv1_w=[]
Spar_conv2_w=[]
Spar_conv1_lin_w=[]
Spar_conv2_lin_w=[]
Global_spar=[]  


# Here is the dictionary to record the list of all measurements
Eva_measure={'base model accuracy':Base_model_accuracy,
            'time inference of base model':T_base_model,
            'number parmameters of base model':Num_parm_base_model,
            'base model size':Base_model_size,
            'energy consumption of base model':Base_Energy_Consumption,
            'cpu usage of base model':Base_Cpu_Usage,
            'memory usage of base model':Base_Memory_Usage,
            'pruned model accuracy': Pruned_model_accuracy,
            'time inference of pruned model':T_pruned_model,
            'number parmameters of pruned model':Num_parm_pruned_model,
            'pruned model size':Pruned_model_size,
            'energy consumption of pruned model':Pruned_Energy_Consumption,
            'cpu usage of pruned model':Pruned_Cpu_Usage,
            'memory usage of pruned model':Pruned_Memory_Usage,
            'pruned finetune model accuracy':Pruned_finetune_model_accuracy,
            'time inference of pruned finetune model':T_pruned_finetune_model,
            'number parmameters of pruned finetune model':Num_parm_pruned_finetune_model,
            'pruned finetune model size':Pruned_finetune_model_size,
            'energy consumption of pruned_finetune model':Pruned_finetune_Energy_Consumption,
            'cpu usage of pruned_finetune model':Pruned_finetune_Cpu_Usage,
            'memory usage of pruned_finetune model':Pruned_finetune_Memory_Usage,
            'Sparsity in conv1.weigh':Spar_conv1_w,
            'Sparsity in conv2.weight' : Spar_conv2_w,
            'Sparsity in conv1.lin.weight' :Spar_conv1_lin_w,
            'Sparsity in conv2.lin.weight':Spar_conv2_lin_w,
            'Global sparsity':Global_spar
            }

### Training, Pruning, Fine-tuning

In [ ]:
for i in range(num_iterations):
    
        print(f'This is iteration {i}')   
        print(f'Training and evaluation before pruning ')
        print("Starting training...")
        Eva=dict()# It is a dictionary to arrange output of this iteration
        best_acc=0  
        early_stopping=100
        is_pruned=False
        save_epoch=10

        #Training base model
        print(f'Training and evaluation before pruning ')
        
        model = Net().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

        for epoch in range(num_epochs):  
            acc=[]
            loss_list = []
            train(callbacks=None)        

            eval_state =  evaluate_Spline(model,data('val_mask'))
            acc_eval=eval_state['acc']
            # report train msg
            if epoch % 20 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")


            # only save the best model
            if eval_state['acc'] > best_acc:
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > early_stopping:
                break
            
            is_best = (eval_state['acc'] > best_acc)
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0

            if is_best or epoch % save_epoch == 0:
                 save_best(ckpt_dir, epoch, model.state_dict(), args.model_name, eval_state['acc'], is_best, is_pruned)   

        #### load the best model
        base_model_path = os.path.join(ckpt_dir, f'{args.model_name}_best.pth') 
        checkpoint = torch.load(base_model_path)
        model.load_state_dict(checkpoint['net'])
        recover_model = lambda: model.load_state_dict(checkpoint['net'])

        # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state
        tracemalloc.start()  # Start tracking memory allocations
        snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        train_acc, base_model_accuracy=test(model)
    

        base_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_base_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        base_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        base_energy_consumption = power_usage * t_base_model
        base_model_size = os.path.getsize(base_model_path)
        num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5) 
        
        print(f'*****Results of base model*********')

        print(f"base model has accuracy on test set={base_model_accuracy:.2f}%")
        print(f"base model has size={base_model_size:.2f} byte")
        print(f"The time inference of base model is ={t_base_model}") 
        print(f"The number of parametrs of base model is:{num_parm_base_model}") 

        print(f"Energy Consumption : {base_energy_consumption:.3f}")
        print(f"total memory usage of base model':{base_total_memory_diff} ")
        print(f"cpu usage of base model':{base_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'base model accuracy': base_model_accuracy,
                    'time inference of base model': t_base_model,
                    'number parmameters of base model': num_parm_base_model,
                    'size of base model': base_model_size, 
                    'energy consumption of base model':base_energy_consumption,
                    'total memory usage of base model':base_total_memory_diff,
                    'cpu usage of base model':base_cpu_usage
                   })

        gc.collect()
        time.sleep(5)  

        print('_________******************************_____________')
        print(f'Pruning the Model')

        recover_model()
        # Parameters for pruning
        parameters_to_prune = [
        (model.conv1, 'weight'),
        (model.conv1.lin, 'weight'),
        (model.conv2, 'weight'),
        (model.conv2.lin, 'weight'),
        ]
        # Pruned model path
        pth_name = f"pruned_model.pth"   
        ckpt_pruned_path = os.path.join(ckpt_dir, pth_name)  
        # Applying pruning method
        pruned_model=global_pruning_torch(model,parameters_to_prune, sparsity, ckpt_pruned_path)
        #Sparsities of layyer after pruning
        spar_dict=get_model_sparsity(pruned_model)
        #recording sparsities of layers
        Eva.update(spar_dict)
        
        # Loading pruned model from disk
        pruned_model=load_and_evaluate_pruned_model( ckpt_pruned_path )
        
        print(f"num_param_pruned:{get_num_parameters(pruned_model, count_nonzero_only=True)}")
        

       
        print('****************Result of pruning ******************')
       
        gc.collect()
        time.sleep(5)  
        tracemalloc.start()  
        snapshot_before = tracemalloc.take_snapshot()
        
        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        train_acc, pruned_model_accuracy=test(pruned_model)
        pruned_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_pruned_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        pruned_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        pruned_energy_consumption = power_usage * t_pruned_model
        pruned_model_size = os.path.getsize(ckpt_pruned_path)
        num_parm_pruned_model=get_num_parameters(pruned_model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  
        
        ###### Report of pruning 
        print(f"pruned model has accuracy on test set={pruned_model_accuracy:.2f}%")
        print(f"pruned model has size={pruned_model_size:.2f} byte")
        print(f"The time inference of pruned model is ={t_pruned_model}") 
        print(f"The number of parametrs of pruned model is:{num_parm_pruned_model}") 

        print(f"Energy Consumption : {pruned_energy_consumption:.3f}")
        print(f"total memory usage of pruned model':{pruned_total_memory_diff} ")
        print(f"cpu usage of pruned model':{pruned_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'pruned model accuracy': pruned_model_accuracy,
                    'time inference of pruned model': t_pruned_model,
                    'number parmameters of pruned model': num_parm_pruned_model,
                    'size of pruned model': pruned_model_size, 
                    'energy consumption of pruned model':pruned_energy_consumption,
                    'total memory usage of pruned model':pruned_total_memory_diff,
                    'cpu usage of pruned model':pruned_cpu_usage
                   })

        gc.collect()
        time.sleep(5)   

        print('________*******************************_____________')
        print(f'Finetuning Pruned Sparse Model')

        best_sparse_checkpoint = dict()
        best_sparse_acc = 0
        is_pruned=True
        
        #### the path of fine-tune model
        fine_tuned_pth_name=f'{args.model_name}_fine_tuned.pth'
        fine_tuned_model_path = os.path.join(ckpt_dir, fine_tuned_pth_name)
        
        #*****************Fine-tuning of model*************************
        for epoch in range(num_epochs):
            # At the end of each train iteration, we have to apply the pruning mask
            #    to keep the model sparse during the training
            acc=[]
            loss_list = []

            acc=train(callbacks=[lambda: global_pruning_torch(model,parameters_to_prune, sparsity,fine_tuned_model_path)])        

            eval_state =  evaluate_Spline(model,data('val_mask'))
            acc_eval=eval_state['acc']

            # report train msg
            if epoch % 20 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")
            
         

            # only save the best model
            if eval_state['acc'] > best_sparse_acc :
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > early_stopping:
                break
            is_best = (eval_state['acc'] > best_sparse_acc )
           
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0
               
  

                           
            #if is_best or epoch % save_epoch == 0:
                
                #Remove Zeroed Weights 
               #sparse_zero_state=state_sparse_model(model, eval_state['acc'])
                #save_best(ckpt_dir, epoch,sparse_zero_state, args.model_name, eval_state['acc'], is_best, is_pruned)
                #sparse_model = load_and_evaluate_pruned_model(fine_tuned_model_path)

  

        # Loading pruned and fine-tuned model from disk
        sparse_model = load_and_evaluate_pruned_model(fine_tuned_model_path)

        # Now we can use the sparse model for evaluation or further training
        print('****************Result of fine-tuning of pruned model ******************')

        gc.collect()
        time.sleep(5)  
        tracemalloc.start() 
        snapshot_before = tracemalloc.take_snapshot()

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)
        train_acc,  pruned_finetune_model_accuracy=test(  sparse_model)
       

        pruned_finetune_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_pruned_finetune_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        pruned_finetune_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        pruned_finetune_energy_consumption = power_usage * t_pruned_finetune_model
        pruned_finetune_model_size = os.path.getsize( fine_tuned_model_path)
        num_parm_pruned_finetune_model=get_num_parameters(sparse_model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state    

        ###### Report  

        print(f"pruned_finetune model has accuracy on test set={pruned_finetune_model_accuracy:.2f}%")
        print(f"pruned_finetune model has size={pruned_finetune_model_size:.2f} byte")
        print(f"The time inference of pruned_finetune model is ={t_pruned_finetune_model}") 
        print(f"The number of parametrs of pruned_finetune model is:{num_parm_pruned_finetune_model}") 

        print(f"Energy Consumption of pruned_finetune model: {pruned_finetune_energy_consumption:.3f}")
        print(f"total memory usage of pruned_finetune model':{pruned_finetune_total_memory_diff} ")
        print(f"cpu usage of pruned_finetune model':{pruned_finetune_cpu_usage:.3f} %")



        #Update my Eva dictionary
        Eva.update({'pruned and finetune model accuracy': pruned_finetune_model_accuracy,
                    'time inference of pruned and finetune model': t_pruned_finetune_model,
                    'number parmameters of pruned and finetune model': num_parm_pruned_finetune_model,
                    'size of pruned and finetune model': pruned_finetune_model_size, 
                    'energy consumption of pruned and finetune model':pruned_finetune_energy_consumption,
                    'total memory usage of pruned and finetune model':pruned_finetune_total_memory_diff,
                    'cpu usage of pruned and finetune model':pruned_finetune_cpu_usage
                   })

        gc.collect()
        time.sleep(5) 


        Base_model_accuracy.append(Eva['base model accuracy'])
        T_base_model.append(Eva['time inference of base model'])
        Num_parm_base_model.append(int(Eva['number parmameters of base model']))
        Base_model_size.append(int(Eva['size of base model']))
        Base_Energy_Consumption.append(Eva['energy consumption of base model'])
        Base_Cpu_Usage.append(Eva['cpu usage of base model'])
        Base_Memory_Usage.append(Eva['total memory usage of base model'])

        Pruned_model_accuracy.append(Eva['pruned model accuracy'])
        T_pruned_model.append(Eva['time inference of pruned model'])
        Num_parm_pruned_model.append(int(Eva['number parmameters of pruned model']))
        Pruned_model_size.append(int(Eva['size of pruned model']))
        Pruned_Energy_Consumption.append(Eva['energy consumption of pruned model'])
        Pruned_Cpu_Usage.append(Eva['cpu usage of pruned model'])
        Pruned_Memory_Usage.append(Eva['total memory usage of pruned model'])


        Pruned_finetune_model_accuracy.append(Eva['pruned and finetune model accuracy'])
        T_pruned_finetune_model.append(Eva['time inference of pruned and finetune model'])
        Num_parm_pruned_finetune_model.append(int(Eva['number parmameters of pruned and finetune model']))
        Pruned_finetune_model_size.append(int(Eva['size of pruned and finetune model']))
        Pruned_finetune_Energy_Consumption.append(Eva['energy consumption of pruned and finetune model'])
        Pruned_finetune_Cpu_Usage.append(Eva['cpu usage of pruned and finetune model'])
        Pruned_finetune_Memory_Usage.append(Eva['total memory usage of pruned and finetune model'])


       
        Spar_conv1_w.append(Eva['conv1.weight'])
        Spar_conv1_lin_w.append(Eva['conv1.lin.weight'])
        Spar_conv2_w.append(Eva['conv2.weight'])
        Spar_conv2_lin_w.append(Eva['conv2.lin.weight'])
        Global_spar.append(Eva['Global sparsity'])     



This is iteration 0
Training and evaluation before pruning 
Starting training...
Training and evaluation before pruning 


### Computing Mean and Std of lists

In [15]:
Eva_final=dict()

base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
Eva_final.update({'Ave of base model size':base_model_size_mean})
Eva_final.update({'Std of base model size':base_model_size_std})


base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})


base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})

base_memory_usage_mean = stat.mean(Base_Memory_Usage)
base_memory_usage_std = stat.stdev(Base_Memory_Usage)
Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})




#################################

pruned_model_accuracy_mean =stat.mean(Pruned_model_accuracy)
pruned_model_accuracy_std = stat.stdev(Pruned_model_accuracy)
Eva_final.update({'Ave of pruned model accuracy':float(format(pruned_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of pruned model accuracy':float(format(pruned_model_accuracy_std, '.3f'))})
                 

t_pruned_model_mean = stat.mean(T_pruned_model)
t_pruned_model_std =stat.stdev(T_pruned_model)
Eva_final.update({'Ave of time inference of pruned model':float(format(t_pruned_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of pruned model':float(format(t_pruned_model_std, '.3f'))})

num_parm_pruned_model_mean = stat.mean(Num_parm_pruned_model)
num_parm_pruned_model_std = stat.stdev(Num_parm_pruned_model)
Eva_final.update({'Ave of number parmameters of pruned model':num_parm_pruned_model_mean})
Eva_final.update({'Std of number parmameters of pruned model':num_parm_pruned_model_std})

pruned_model_size_mean =stat.mean( Pruned_model_size)
pruned_model_size_std = stat.stdev(Pruned_model_size)
Eva_final.update({'Ave of pruned model size':pruned_model_size_mean})
Eva_final.update({'Std of pruned model size':pruned_model_size_std })

pruned_energy_consumption_mean = stat.mean(Pruned_Energy_Consumption)
pruned_energy_consumption_std = stat.stdev(Pruned_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of pruned model':pruned_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of pruned model':pruned_energy_consumption_std})


pruned_cpu_usage_mean = stat.mean(Pruned_Cpu_Usage)
pruned_cpu_usage_std = stat.stdev(Pruned_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of pruned model':pruned_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of pruned model':pruned_cpu_usage_std})

pruned_memory_usage_mean = stat.mean(Pruned_Memory_Usage)
pruned_memory_usage_std = stat.stdev(Pruned_Memory_Usage)
Eva_final.update({'Ave of memory usage of pruned model':pruned_memory_usage_mean})
Eva_final.update({'Std of memory usage of pruned model':pruned_memory_usage_std})


#################################
pruned_finetune_model_accuracy_mean =stat.mean(Pruned_finetune_model_accuracy)
pruned_finetune_model_accuracy_std = stat.stdev(Pruned_finetune_model_accuracy)
Eva_final.update({'Ave of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_std, '.3f'))})                 

t_pruned_finetune_model_mean =stat.mean(T_pruned_finetune_model)
t_pruned_finetune_model_std =stat.stdev(T_pruned_finetune_model)
Eva_final.update({'Ave of time inference of pruned finetune model':float(format(t_pruned_finetune_model_mean,'.3f'))})
Eva_final.update({'Std of time inference of pruned finetune model':float(format(t_pruned_finetune_model_std,'.3f'))})

num_parm_pruned_finetune_model_mean =stat.mean(Num_parm_pruned_finetune_model)
num_parm_pruned_finetune_model_std = stat.stdev(Num_parm_pruned_finetune_model)
Eva_final.update({'Ave of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_mean})
Eva_final.update({'Std of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_std })

pruned_finetune_model_size_mean = stat.mean(Pruned_finetune_model_size)
pruned_finetune_model_size_std = stat.stdev(Pruned_finetune_model_size)
Eva_final.update({'Ave of pruned finetune model size':pruned_finetune_model_size_mean})
Eva_final.update({'Std of pruned finetune model size':pruned_finetune_model_size_std})


pruned_finetune_energy_consumption_mean = stat.mean(Pruned_finetune_Energy_Consumption)
pruned_finetune_energy_consumption_std = stat.stdev(Pruned_finetune_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_std})


pruned_finetune_cpu_usage_mean = stat.mean(Pruned_finetune_Cpu_Usage)
pruned_finetune_cpu_usage_std = stat.stdev(Pruned_finetune_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_std})

pruned_finetune_memory_usage_mean = stat.mean(Pruned_finetune_Memory_Usage)
pruned_finetune_memory_usage_std = stat.stdev(Pruned_finetune_Memory_Usage)
Eva_final.update({'Ave of memory usage of pruned_finetune model':pruned_finetune_memory_usage_mean})
Eva_final.update({'Std of memory usage of pruned_finetune model':pruned_finetune_memory_usage_std})



sparsity_conv1_w_mean = stat.mean(Spar_conv1_w)
sparsity_conv1_w_std = stat.stdev(Spar_conv1_w)
Eva_final.update({'Sparsity in conv1.weight':float(format(sparsity_conv1_w_mean,'.3f'))})
Eva_final.update({'Std of Sparsity in conv1.weight':float(format(sparsity_conv1_w_std,'.3f'))})

sparsity_conv1_lin_w_mean = stat.mean( Spar_conv1_lin_w)
sparsity_conv1_lin_w_std = stat.stdev( Spar_conv1_lin_w)
Eva_final.update({'Sparsity in conv1.lin.weight':float(format(sparsity_conv1_lin_w_mean,'.3f'))})
Eva_final.update({'Std of Sparsity in conv1.lin.weight':float(format(sparsity_conv1_lin_w_std,'.3f'))})

sparsity_conv2_w_mean = stat.mean(Spar_conv2_w)
sparsity_conv2_w_std = stat.stdev(Spar_conv2_w)
Eva_final.update({'Sparsity in conv2.weight':float(format(sparsity_conv2_w_mean,'.3f'))})
Eva_final.update({'Std of Sparsity in conv2.weight':float(format(sparsity_conv2_w_std,'.3f'))})

sparsity_conv2_lin_w_mean = stat.mean( Spar_conv2_lin_w)
sparsity_conv2_lin_w_std = stat.stdev( Spar_conv2_lin_w)
Eva_final.update({'Sparsity in conv2.lin.weight':float(format(sparsity_conv2_lin_w_mean,'.3f'))})
Eva_final.update({'Std of Sparsity in conv2.lin.weight':float(format(sparsity_conv2_lin_w_std,'.3f'))})


Global_sparsity_mean = stat.mean(Global_spar)
Global_sparsity_std = stat.stdev(Global_spar)
Eva_final.update({'Global sparsity':float(format(Global_sparsity_mean,'.3f'))})
Eva_final.update({'Std of Global sparsity':float(format(Global_sparsity_std,'.3f'))})


#################################


print(f"All measurement about pruning process of sparsity:{sparsity*100}% ")   
Eva_final

All measurement about pruning process of sparsity:90.0% 


{'Ave of base model accuracy': 0.878,
 'Std of base model accuracy': 0.006,
 'Ave of time inference of base model': 27.581,
 'Std of time inference of base model': 0.947,
 'Ave of number parmameters of base model': 69143,
 'Std of number parmameters of base model': 0.0,
 'Ave of base model size': 279991,
 'Std of base model size': 0.0,
 'Ave of energy consumption of base model': 816.1701240162686,
 'Std of energy consumption of base model': 185.40925873035033,
 'Ave of cpu usage of base model': 71.6,
 'Std of cpu usage of base model': 8.626702730475882,
 'Ave of memory usage of base model': 10384,
 'Std of memory usage of base model': 4027.6802256385745,
 'Ave of pruned model accuracy': 0.874,
 'Std of pruned model accuracy': 0.011,
 'Ave of time inference of pruned model': 26.558,
 'Std of time inference of pruned model': 3.287,
 'Ave of number parmameters of pruned model': 6935,
 'Std of number parmameters of pruned model': 0.0,
 'Ave of pruned model size': 171787,
 'Std of pruned_mo

### Recording results on txt file

In [18]:


dataset_name = 'Cora'
Pruning_Method='Global_Pruning'
max_epoch = 10
resume = True
result_folder ='pathresult/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)



file_name = result_folder+Pruning_Method+'_'+'with sparsity of'+'_'+str(sparsity)+'_on_'+dataset_name+'_'+'Node classification task'+'_'+str(max_epoch)+'.txt'
if not os.path.exists(file_name):
        with open(file_name, 'w') as f:
                f.write('%s:%s\n'%('dataset_name', 'Cora'))
                f.write('%s:%s\n'%('Task', 'Node Classification'))
                f.write('%s:%s\n'%('max_epoch', max_epoch))
                f.write('%s:%s\n'%('sparsity', sparsity))
                for key, value in Eva_final.items():
                    f.write('%s:%s\n'%(key, value))
                    
                for key, value in Eva_measure.items():
                    f.write('%s:%s\n' % (key, ','.join(map(str, value))))                            
       